# Experiments

---
## Model Setup

In [2]:
from huggingface_hub import InferenceClient
from transformers import AutoTokenizer
from openai import OpenAI

# Name models
GPT_MODEL_NAME = "gpt-3.5-turbo-1106"
CODELLAMA_MODEL_NAME = "codellama/CodeLlama-70b-Instruct-hf"

# Get API keys from the .env file
OPENAI_API_KEY = open(".env", "r").read().split("\n")[0]
HUGGINGFACE_API_KEY = open(".env", "r").read().split("\n")[1]


# Set up the OpenAI API client
gpt_client = OpenAI(api_key=OPENAI_API_KEY)

# Set up the codellama API client
codellama_client = InferenceClient(
    model=CODELLAMA_MODEL_NAME, token=HUGGINGFACE_API_KEY
)

# Set up codellama tokenizer
tokenizer = AutoTokenizer.from_pretrained(CODELLAMA_MODEL_NAME)

c:\Users\jaspe\Documents\MLAssignment2\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


## Prompt Interface Setup

In [3]:
def prompt_model(prompt, model=GPT_MODEL_NAME):
    chat = [
        {"role": "user", "content": prompt},
    ]

    if model == GPT_MODEL_NAME:

        response = str(
            gpt_client.chat.completions.create(model=GPT_MODEL_NAME, messages=chat)
            .choices[0]
            .message.content
        )

    elif model == CODELLAMA_MODEL_NAME:

        response = codellama_client.text_generation(
            tokenizer.apply_chat_template(chat, tokenize=False),
            max_new_tokens=150,
            stop_sequences=["<step>", "</s>"],
        )

    return response

## Testing Basic Operation

In [4]:
test_prompt = "Write a simple Python multiplication function"

print("CODELLAMA:\n" + prompt_model(test_prompt, model=CODELLAMA_MODEL_NAME))


print("GPT:\n" + prompt_model(test_prompt, model=GPT_MODEL_NAME))

CODELLAMA:

```
def multiply(a, b):
    return a * b
```
This function takes two numbers as input and returns their product. <step>
GPT:
def multiply(a, b):
    return a * b

# Example usage
result = multiply(4, 5)
print(result)  # Output: 20


---
## E1: Function Name Generation

In [7]:
import re

base_prompt = "Recommend a function name for the following code: {code}\n"

open("results/E1_results.csv", "w", encoding="utf-8").write("")

for function in open("test_data/test_data.csv", "r").read().split("\n"):
    data = function.split(",")
    difficulty = data[0]
    function_name = data[1]
    source_code = data[2:]
    gpt_response = (
        prompt_model(base_prompt.format(code=source_code), model=GPT_MODEL_NAME)
        .replace('"', "")
        .replace(")", "")
        .replace("(", "")
        .replace("\n", "")
    )
    full_codellama_response = (
        prompt_model(base_prompt.format(code=source_code), model=CODELLAMA_MODEL_NAME)
        .replace('"', "")
        .replace(")", "")
        .replace("(", "")
        .replace("\n", "")
    )
    # Use regex to extract function name. This is done by looking for text in first pair of ""
    codellama_response = re.findall(r'"([^"]*)"', full_codellama_response)
    if len(codellama_response) > 0:
        codellama_response = codellama_response[0]
    else:
        codellama_response = "ERROR: CLEAN THIS"
    output_file = open("results/E1_results.csv", "a", encoding="utf-8")
    output_file.write(
        f"{difficulty},{function_name},{gpt_response},{codellama_response},{full_codellama_response}\n".replace(
            "(", ""
        ).replace(
            ")", ""
        )
    )
    output_file.close()
    print(
        f"Actual: {function_name}, GPT: {gpt_response}, CodeLlama: {codellama_response}"
    )

Actual: multiply, GPT: multiplyNumbers, CodeLlama: multiply
Actual: add, GPT: addNumbers, CodeLlama: add_numbers
Actual: subtract, GPT: subtractNumbers, CodeLlama: subtract
Actual: divide, GPT: calculateDivision, CodeLlama: divide
Actual: power, GPT: raiseToPower, CodeLlama: power
Actual: modulus, GPT: divide_remainder, CodeLlama: modulo
Actual: floor_divide, GPT: divideNumbers, CodeLlama: floor_divide
Actual: equals, GPT: is_equal, CodeLlama: is_equal
Actual: not_equals, GPT: isNotEqual, CodeLlama: are_not_equal
Actual: greater_than, GPT: compare_numbers, CodeLlama: isGreaterThan
Actual: less_than, GPT: compareNumbers, CodeLlama: is_less_than
Actual: greater_than_or_equal, GPT: compareNumbers, CodeLlama: is_greater_than_or_equal_to
Actual: less_than_or_equal, GPT: isLessThanOrEqualTo, CodeLlama: is_less_than_or_equal_to
Actual: logical_and, GPT: multiply_values, CodeLlama: and_operation
Actual: logical_or, GPT: choose_non_null_value, CodeLlama: get_first_non_falsey_value
Actual: logic

---
## E2: Documentation Generation

In [18]:
base_prompt = "Write a documentation paragraph for the following function: {code}\n"

open("results/E2_results.csv", "w", encoding="utf-8").write("")

for function in open("test_data/test_data.csv", "r").read().split("\n"):
    data = function.split(",")
    difficulty = data[0]
    function_name = data[1]
    source_code = "".join(data[2:])
    gpt_response = (
        prompt_model(base_prompt.format(code=source_code), model=GPT_MODEL_NAME)
        .replace(",", "")
        .replace("\n", "")
    )
    codellama_response = (
        prompt_model(base_prompt.format(code=source_code), model=CODELLAMA_MODEL_NAME)
        .replace(",", "")
        .replace("\n", "")
    )
    output_file = open("results/E2_results.csv", "a", encoding="utf-8")
    output_file.write(
        f"{difficulty},{function_name},{gpt_response},{codellama_response},{source_code}\n"
    )
    output_file.close()
    print(
        f"Function Name: {function_name}, GPT: {gpt_response}, CodeLlama: {codellama_response}"
    )

Function Name: multiply, GPT: The function returns the product of two input parameters a and b. It takes in two numerical values as arguments and multiplies them together to get the result. This function is a simple multiplication operation and does not handle any special cases or error checking. The return value will be the product of the two input parameters., CodeLlama: Here is a documentation paragraph for the function:```/** * Multiplies two numbers and returns the result. * * @param {number} a - The first number to multiply. * @param {number} b - The second number to multiply. * @returns {number} The result of multiplying a and b. */function multiply(a b) {    return a * b;}```This function takes two numbers `a` and `b` as input and multiplies them together. It then returns the result of the multiplication. The function is documented with a brief description of what it does as well as the parameters it takes
Function Name: add, GPT: The `return a + b` function takes two numerical

---
## E3: Code Generation

In [17]:
base_prompt = "Given the following function name generate python code: {code}\n"

open("results/E3_results.csv", "w", encoding="utf-8").write("")

for function in open("test_data/test_data.csv", "r").read().split("\n"):
    data = function.split(",")
    difficulty = data[0]
    function_name = data[1]
    source_code = data[2:]
    gpt_response = (
        prompt_model(base_prompt.format(code=function_name), model=GPT_MODEL_NAME)
        .replace(",", "")
        .replace("\n", "")
    )
    codellama_response = (
        prompt_model(base_prompt.format(code=function_name), model=CODELLAMA_MODEL_NAME)
        .replace(",", "")
        .replace("\n", "")
    )
    codellama_response_code = re.findall(r"```([^`]*)```", codellama_response)
    if len(codellama_response_code) > 0:
        codellama_response_code = codellama_response_code[0]
    else:
        codellama_response_code = "ERROR: CLEAN THIS"
    output_file = open("results/E3_results.csv", "a", encoding="utf-8")
    output_file.write(
        f"{difficulty},{function_name},{gpt_response},{codellama_response_code},{codellama_response},{source_code}\n"
    )
    output_file.close()
    print(
        f"Function Name: {function_name}, GPT: {gpt_response}, CodeLlama: {codellama_response_code}"
    )

Function Name: multiply, GPT: def multiply(a b):    return a * b, CodeLlama: def multiply(a b):    return a * b
Function Name: add, GPT: def add(x y):    return x + y, CodeLlama: def add(a b):    return a + b
Function Name: subtract, GPT: def subtract(num1 num2):    return num1 - num2, CodeLlama: def subtract(a b):    return a - b
Function Name: divide, GPT: def divide(num1 num2):    return num1 / num2, CodeLlama: def divide(a b):    return a / b
Function Name: power, GPT: def power(base exponent):    return base ** exponent, CodeLlama: def power(base exponent):    result = 1    for _ in range(exponent):        result *= base    return result
Function Name: modulus, GPT: def modulus(num1 num2):    return num1 % num2, CodeLlama: pydef modulus(a b):    return a % b
Function Name: floor_divide, GPT: def floor_divide(num1 num2):    return num1 // num2, CodeLlama: pythondef floor_divide(a b):    return a // b
Function Name: equals, GPT: def equals(a b):    if a == b:        return True    e